In [ ]:
#rough idea of hardware->ML pipeline for real time, still a work in progress
#also includes basic ML->unity keystroke


import serial
import re
import numpy as np
from collections import deque

#this is importing the ML algos, could also export using pickle
import joblib 

#config
SERIAL_PORT = 'COM8'
BAUD_RATE = 9600
WINDOW_SIZE = 50  # number of samples per inference
MODEL_PATH = "my_model.pkl"  # trained classifier

#load ML model
model = joblib.load(MODEL_PATH222)

#regex patterns
gyro_pattern = re.compile(r'Gyro\. X = (-?\d+), Y = (-?\d+), Z = (-?\d+)')
acc_pattern  = re.compile(r'Acc\. X = (-?\d+), Y = (-?\d+), Z = (-?\d+)')

#serial connection
ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)

#buffer for real-time pipeline
buffer = deque(maxlen=WINDOW_SIZE)

print("Streaming data... Press Ctrl+C to stop.")

try:
    while True:
        line = ser.readline().decode('utf-8', errors='ignore').strip()

        gyro_match = gyro_pattern.match(line)
        if gyro_match:
            gyro_data = list(map(int, gyro_match.groups()))
            
            acc_line = ser.readline().decode('utf-8', errors='ignore').strip()
            acc_match = acc_pattern.match(acc_line)

            if acc_match:
                acc_data = list(map(int, acc_match.groups()))
                sample = gyro_data + acc_data  # [gx, gy, gz, ax, ay, az]
                buffer.append(sample)

                #once we have enough samples, run inference
                if len(buffer) == WINDOW_SIZE:
                    window = np.array(buffer)
                    # TODO: apply preprocessing here
                    features = extract_features(window)  
                    prediction = model.predict([features])[0]
                    confidence = max(model.predict_proba([features])[0])

                    if confidence > 0.7:
                        print("Predicted action:", prediction)
                        # TODO: send_to_unity(prediction)
                    else:
                        print("No confident action detected")
except KeyboardInterrupt:
    print("\nStopped streaming.")
finally:
    ser.close()
